This is a code in an unofficial walkthrough of the Expectation Maximization algorithm in Figure 1 of

Do, C. B., & Batzoglou, S. (2008). What is the expectation maximization algorithm? Nature Biotechnology, 26(8), 897–899. http://doi.org/10.1038/nbt1406

I wanted to better understand what exactly the expectation maximization algorithm was doing at an intuitive level, as most of the abstract and purely mathematical explanations I had watched on youtube or read online weren't clicking. I've always found there is no better way to learn something than to try it until it works and makes sense.

If you have access to the original paper, I highly recommend reading it, as I'm sure this will make much more sense after reading the authors original words and explanation

![](header.png)

And, as mentioned earlier, we will focus on the lone figure of the paper, and work through the iterations step by step. Here is the full figure. Try and walk through it mentally, and see which parts you can understand directly from the figure, and make a mental note of which steps don't make sense. During the walk through, come back to these original ideas and see which parts you were correct on, and see if the code helps the unclear steps make sense

![](Figure1_with_caption.png)

In [ ]:
using DataArrays, DataFrames
using HypothesisTests
using Distributions
using Formatting

### The Observed Data

Here is the table from Figure 1 that shows the true underlying data.
![](coin_table.png)

If we could observe which coin was used for each round, we could compute the true probability of each coin
![](true_data.png)

However, in the event we are unable to know which coin was used in each trial, we would like to approximate the maximum likelihood probabilities of the two coins

Let's set up a DataFrame in Julia that represents this

In [ ]:
Data = DataFrame(
    COIN = ["B", "A", "A", "B", "A"],
    FLIPS = ["HTTTHHTHTH", "HHHHTHHHHH", "HTHHHHHTHH", "HTHTTTHHTT", "THHHTHHHTH"]
)

We have included the true state of the coin, however, we will blind ourselves to that knowledge and proceed as if we could not observe true state of each coin used. We can however observe the resulting flip data. Let's go ahead and precompute the number of heads and tails for each trial for easier calculations later. I'll break down the steps first though.

The countmap function takes an iterable, like a list, and counts the number of occurances of each unique value in the iterable.

Our trials are currently in the form of a string

In [ ]:
Data[:FLIPS][1]

Now let's turn it into a list by taking a list comprehension on it.

The ' operator on the end is the transpose operator. It simply coverts the array from long, columnar form to wide, row form and makes it cleaner to print to the console

In [ ]:
[flip for flip in Data[:FLIPS][1]]'

now we can wrap that in a countmap call to get a dictionary of counts for heads and tails

In [ ]:
countmap([flip for flip in Data[:FLIPS][1]])

and to get the numbers of heads and tails, we just need to do a dictionary look up with `['H']` and `['T']`

In [ ]:
println(countmap([flip for flip in Data[:FLIPS][1]])['H'])
println(countmap([flip for flip in Data[:FLIPS][1]])['T'])

now we are ready to do the same steps as above, but use the map function to perform the calculations on every trial in the experiment, which we can do with the map function

In [ ]:
Data[:HEADS] = map((x) -> countmap([i for i in x])['H'], Data[:FLIPS]);
Data[:TAILS] = map((x) -> countmap([i for i in x])['T'], Data[:FLIPS]);

Here is the Data with precomupted heads and tails

In [ ]:
Data

Here are the values from the figure for comparison

![](heads_tails.png)

For a santiy check, let's compute the true heads probabilities of each of the coins to confirm that they match what is shown in the figure. To do this, we will compute the probabiities by computing over grouped DataFrames

Here we group the DataFrame `Data` by the values in the column `COIN`, and then on each of the resulting sub DataFrames (one each for coin A and coin B), we sum the total number of heads of all trials for the given coin, and then divide that by the total number of flips for the given coin to calculate the head probability for each coin. Again, confirm this matches the probabilities in the figure

In [ ]:
by(Data, :COIN, df -> sum(df[:HEADS]) / (sum(df[:HEADS]) + sum(df[:TAILS])))

![](original_data.png)

Great! Now we are set up to start the algorithm. Before we can start, we need to choose initial values for the probabilties. Do & Batzoglou 2008 used $θ_A = 0.6$ and $θ_B = 0.5$

![](initial_probabilities.png)

We could however, choose any initial starting values. We will follow the example in the paper first. Later, I will try and use randomly drawn initial values to see if we converge to the same values

In [ ]:
θA_init = 0.6;
θB_init = 0.5;

Now that we have our initial states and our data, we can now evaluate the relative probabilities that each trial of 10 coin flips was generated by coin A or coin B.

Again, assuming that the trails were produced by an unknown coin, our data now looks like this
![](unassigned_coins.png)

We want to assign a relative probability to each of those trials being generated by each of the possible states (coins)

Here is our now unlabeled data
![](trial_1.png)

And let's see how we can grab the precomputed # of heads

In [ ]:
Data[1, :HEADS]

and tails

In [ ]:
Data[1, :TAILS]

Now we can think of total number of flips as being a generic total number of trials. In the same manner, we can think of the number of heads as being a generic number of successes. 

In [ ]:
successes = Data[1, :HEADS]
trials = Data[1, :HEADS] + Data[1, :TAILS]
println("$successes\t$trials")

In this format, we can now plug the data into a generic binomial likelihood formula to calculate the likelihood of observing the given data.

The formula is

$$ {n \choose k}\theta^k (1-\theta)^{n-k} $$

where
$$ n = \text{# trails} $$
$$ k = \text{# successes} $$
$$ \theta = \text{likelihood of getting a heads for a given coin} $$

In [ ]:
# θA
theta = θA_init
A_likelihood = binomial(trials, successes) * theta^successes * (1-theta)^(trials - successes)
println("probability of observing this data with coin A $A_likelihood")
# θB
theta = θB_init
B_likelihood = binomial(trials, successes) * theta^successes * (1-theta)^(trials - successes)
println("probability of observing this data with coin B $B_likelihood")

If you refer back to the figure, you'll notice that these values aren't the values assigned to coin A and coin B for trial one. Those values are:

![](trial_1_probabilities.png)

Have we done something wrong, or are we simply not quite there? I wasn't sure at first, but let's make an intuitive guess. In general, statistical tests always require that your total probability sums to 1. Let's try normalizing our data. Given that we have the probability that this data was generated by fictional coin A and the probability that this data was generated by fictional coin B, we can calculate the total probability that this data was generated by either coin

In [ ]:
total_likelihood = A_likelihood + B_likelihood

This still doesn't sum to 1, but if we normalize our data to this total, then our updated probabilities, which will now be relative to the total probability, should sum to 1

In [ ]:
A_likelihood = A_likelihood / total_likelihood
B_likelihood = B_likelihood / total_likelihood
println("$A_likelihood x A\t$B_likelihood x B")

That looks better! And sum to confirm

In [ ]:
A_likelihood + B_likelihood

and with simple rounding we arrive at the same numbers as the authors

In [ ]:
println("$(fmt(".2f","$A_likelihood")) x A  $(fmt(".2f","$B_likelihood")) x B")

And we match!
![](trial_1_probabilities.png)

Now that we have calculated the relative likelihood tha each of the two made up coins produced the observed data, let's go ahead and assume that the two made up coins actually did generate this data. We will say that each of the two coins contributed `h` heads and `t` tails. To arrive at this number for each coin, simply multiply the relative probability of each coin by the actual observed numbers for heads and tails of the data

In [ ]:
A_heads = Data[1, :HEADS] * A_likelihood;
A_tails = Data[1, :TAILS] * A_likelihood;
B_heads = Data[1, :HEADS] * B_likelihood;
B_tails = Data[1, :TAILS] * B_likelihood;

And format the values in a DataFrame for easier viewing

In [ ]:
DataFrame(Coin_A = "=$(fmt(".1f","$A_heads")), H $(fmt(".1f","$A_tails")) T",
          Coin_B = "=$(fmt(".1f","$B_heads")), H $(fmt(".1f","$B_tails")) T")

![](trial_1_heads_tails_relative_blame.png)

Now we are correctly computing the relative contribution of heads and tails for each coin in terms of generating the observed data. 

This was originally very unintuitive to me, because we know from the original data which coin was actually used to generate each data set. Why would we split responsibility across the two coins? That is a mathematical requirement of the optimization I don't understand. However we can confirm that this is the right approach. [Here is an overview video of the expectation maximization algorithm by Pavel Pevzner from UCSD](https://youtu.be/P1r4RR1goIU?list=PLQ-85lQlPqFMcC2d2CkvmdcJt2v-Np7Cz). I recommend watching the whole playlist of videos, but for the purpose of this tutorial, it's probabily sufficient to watch only the video directly linked to

Now let's write a function that computes the relative responsilibity of a generic row with two generic probabilities. This could be extended to accomidate any number of coins though by passing a list of θ's rather than θ1, θ2

In [ ]:
function assign_blame(row, θ1, θ2)
    successes = row[:HEADS]
    trials = row[:HEADS] + row[:TAILS]
    probabilities = []
    for θ in [θ1, θ2]
        push!(probabilities, binomial(trials, successes) * θ^successes * (1-θ)^(trials - successes))
    end
    total = sum(probabilities)
    probabilities = [p/total for p in probabilities]
    return probabilities
end

And now we will map that function onto each row/trial in our dataframe, using the initial guessed probabilities for each of the two coins.

In [ ]:
# map the generic input x to the fxn assign blame, passing x, the initial θA and intial θB, passing each row of the
# DataFrame Data as the input x
output = map((x) -> assign_blame(x, θA_init, θB_init), eachrow(Data))
# output is in the form of an array of arrays. We can use the ... splat operator to dump the array of arrays
# and hcat to merge them into a matrix. However, the generated matrix is a 2x5 matrix, and we want a 5x2 matrix
# so take the transpose with the ' operator
blame = hcat(output...)'

And for the sake of clarity, let's format this to match the format shown in the figure

In [ ]:
for i = 1:size(blame, 1)
    println("$(fmt(".2f", blame[i,1])) x A\t$(fmt(".2f", blame[i,2])) x B")
end

![](relative_blame.png)

Now that we have the relative responsibilites (blame) of each coin, now we count up how many heads and tails each coin deserves based on the level of blame it has. Then recompute probabilities for each coin

In [ ]:
Aheads = Data[:HEADS] .* blame[:,1];
Atails = Data[:TAILS] .* blame[:,1];
CoinA = hcat(Aheads, Atails)

In [ ]:
Bheads = Data[:HEADS] .* blame[:,2];
Btails = Data[:TAILS] .* blame[:,2];
CoinB = hcat(Bheads, Btails)

Again, let's clean it up for the sake of example

In [ ]:
relative_blame = DataFrame(Coin_A = ["=$(fmt(".1f", CoinA[i,1])) H, $(fmt(".1f", CoinA[i,2])) T" for i = 1:size(CoinA, 1)],
        Coin_B = ["=$(fmt(".1f", CoinB[i,1])) H, $(fmt(".1f", CoinB[i,2])) T" for i = 1:size(CoinB, 1)])

![](relative_heads_and_tails.png)

Great! However, if we want to update our guess about the maximum likelihood probability of each of our made up coins, we need to calculate

$$ \hat{\theta}_A = \frac{\text{total heads coin A}}{\text{total heads + total tails coin A}}$$

In [ ]:
CoinATotals = "=$(fmt(".1f", sum(Aheads))) H, $(fmt(".1f", sum(Atails))) T"
CoinBTotals = "=$(fmt(".1f", sum(Bheads))) H, $(fmt(".1f", sum(Btails))) T"
TotalsRow = DataFrame(Coin_A = CoinATotals, Coin_B = CoinBTotals)
vcat(relative_blame, TotalsRow)

And again, we match
![](relative_blame_with_totals.png)

We're now all set to compute the values, so let's get on with it

In [ ]:
θA_new = sum(Aheads)/sum(Aheads + Atails)
θB_new = sum(Bheads)/sum(Bheads + Btails)
println("new θA = $(fmt(".2f", θA_new))")
println("new θB = $(fmt(".2f", θB_new))")

![](recalculate.png)

That's a match! Now, let's put out thetas into a DataFrame to keep track of our iterations

In [ ]:
Thetas = DataFrame(θA = θA_new, θB = θB_new)

And there you have it. You've successfully walked through the first iteration of the expectation maximization algorithm as it applies to the example in Figure 1 of Do & Batzoglou 2008!

However, the true power of the expectation maximization algorithm is the result of it's iterative potential. Normally, this algorithm will be run until a certain level of convergence is reached. In practice, this can be set to some level of change, such as when your updated thetas change by less than .001 between iterations. Do & Batzoglou 2008 show final output after 10 iterations, so let's go with that

![](convergence.png)

In [ ]:
for i = 2:10
    # returns an array of arrays
    output = map((x) -> assign_blame(x, Thetas[end, :θA], Thetas[end, :θB]), eachrow(Data));
    # cast the array of arrays into a 5x2 matrix
    blame = hcat(output...)';
    
    Aheads = Data[:HEADS] .* blame[:,1];
    Atails = Data[:TAILS] .* blame[:,1];
    
    Bheads = Data[:HEADS] .* blame[:,2];
    Btails = Data[:TAILS] .* blame[:,2];
    
    θA_new = sum(Aheads)/sum(Aheads + Atails)
    θB_new = sum(Bheads)/sum(Bheads + Btails)
    
    Thetas = vcat(Thetas, DataFrame(θA = θA_new, θB = θB_new))
end

And let's see what our final thetas are after 10 iterations by grabbing θA and θB from the Thetas DataFrame at row 10. Remember we are appending each successive iteration as a new row

In [ ]:
println("θA 10 = $(fmt(".2f", Thetas[10, :θA]))")
println("θB 10 = $(fmt(".2f", Thetas[10, :θB]))")

Answers match!

![](tenth_iteration.png)

And that's a walkthrough of the entire Expectation Maximization algorithm as it applies to Figure 1 from Do & Batzoglou 2008

![](Figure1.png)

Next steps:
1. try starting from randomly seeded start probabilities
1. implement a dynamic cut off that terminates the EM iterations when we see reduced improvement to the point that we claim convergence

In [ ]:
# θA = rand(Uniform(0, 1));
# θB = rand(Uniform(0, 1));